##### temp

In [12]:
import torch
import transformers
import sentence_transformers
import sklearn
import numpy
import scipy

print(f"torch: {torch.__version__}")
print(f"transformers: {transformers.__version__}")
print(f"sentence-transformers: {sentence_transformers.__version__}")
print(f"scikit-learn: {sklearn.__version__}")
print(f"numpy: {numpy.__version__}")
print(f"scipy: {scipy.__version__}")

torch: 2.2.2
transformers: 4.54.1
sentence-transformers: 5.0.0
scikit-learn: 1.7.1
numpy: 2.3.2
scipy: 1.16.1


추천조합

pip install torch==2.0.1
#pip install transformers==4.30.2
pip install sentence-transformers==2.2.2
pip install scikit-learn==1.3.2
pip install numpy==1.24.4
pip install scipy==1.10.1

pip install transformers==4.25.1
pip install huggingface-hub==0.10.1

이어서 이대로 다시 깔기

##### temp end

In [7]:
import torch
import transformers
import sentence_transformers
import sklearn
import numpy
import scipy

print(f"torch: {torch.__version__}")
print(f"transformers: {transformers.__version__}")
print(f"sentence-transformers: {sentence_transformers.__version__}")
print(f"scikit-learn: {sklearn.__version__}")
print(f"numpy: {numpy.__version__}")
print(f"scipy: {scipy.__version__}")

torch: 2.2.2
transformers: 4.25.1
sentence-transformers: 2.2.2
scikit-learn: 1.3.2
numpy: 1.24.4
scipy: 1.10.1


In [8]:
from sentence_transformers import SentenceTransformer
import numpy as np

# 1단계: 문장 임베딩
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

reviews = [
    "배송이 빠르고 제품도 좋아요.",
    "품질이 기대 이상입니다!",
    "생각보다 배송이 오래 걸렸어요.",
    "배송은 느렸지만 포장은 안전했어요.",
    "아주 만족스러운 제품입니다."
]

embeddings = model.encode(reviews)

In [9]:
import psycopg2

# 2단계: PostgreSQL 테이블 생성
conn = psycopg2.connect(
    host="localhost",          # 기본값 그대로 사용
    port=5432,                 # PostgreSQL 기본 포트
    database="postgres",       # 또는 직접 만든 DB명 (예: reviewdb)
    user="postgres",           # PostgreSQL 기본 사용자
    password="0000"            # 당신이 설정한 비밀번호
)

cur = conn.cursor()

cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
cur.execute("""
    CREATE TABLE IF NOT EXISTS review_vectors (
        id SERIAL PRIMARY KEY,
        review TEXT,\
        embedding VECTOR(384)
    );
""")
conn.commit()

In [10]:
# 3단계: 벡터 저장

for review, emb in zip(reviews, embeddings):
    emb_list = emb.tolist()
    cur.execute(
        "INSERT INTO review_vectors (review, embedding) VALUES (%s, %s)",
        (review, emb_list)
    )

conn.commit()

In [11]:
# 4단계: 유사도 검색

query = "배송이 느렸어요"
query_vec = model.encode([query])[0].tolist()

#cur.execute(
#    """
#    SELECT review, embedding <=> %s AS distance
#    FROM review_vectors
#    ORDER BY embedding <=> %s
#    LIMIT 3;
#    """,
#    (query_vec, query_vec)
#)

cur.execute(
    """
    SELECT review, embedding <=> CAST(%s AS vector) AS distance
    FROM review_vectors
    ORDER BY embedding <=> CAST(%s AS vector)
    LIMIT 3;
    """,
    (query_vec, query_vec)
)

results = cur.fetchall()
for r in results:
    print(f"유사도: {r[1]:.4f} | 리뷰: {r[0]}")

유사도: 0.0783 | 리뷰: 배송이 빠르고 제품도 좋아요.
유사도: 0.0783 | 리뷰: 배송이 빠르고 제품도 좋아요.
유사도: 0.0990 | 리뷰: 배송은 느렸지만 포장은 안전했어요.


In [12]:
# 5. 마무리

cur.close()
conn.close()

🔁 참고: 유사도 계산 연산자

<=> : pgvector에서 제공하는 L2 거리 (Euclidean) 또는 Cosine 유사도 기반 정렬을 자동 적용

embedding <=> %s : 유사한 순서대로 오름차순 정렬됨 (작을수록 유사)
